In [1]:
HOST = "10.42.160.9"
USER = "postgres"
PW = "Austin2023"
DB = "postgres"

In [2]:
import psycopg

def connect():
    
    url = "dbname='{}' user='{}' host={} port=5432 password={}".format(DB, USER, HOST, PW)
    connection = psycopg.connect(url)

    return connection

#### 1. Test Connection 

In [3]:
import psycopg

connection = connect()

print(connection)

<psycopg.Connection [IDLE] (host=10.42.160.9 database=postgres) at 0x7fbcbe07f5b0>


#### 2. Create the table (and drop it if it already exists)

In [4]:
import psycopg

connection = connect()
print(connection)

drop = 'DROP TABLE IF EXISTS reservations'

create = '''CREATE TABLE reservations(
    res_id SERIAL PRIMARY KEY,
    cust_id INTEGER NOT NULL,
    prp_nm VARCHAR(50) NOT NULL,
    prp_ch VARCHAR(50) NOT NULL,
    adr_line_1 VARCHAR(40) NOT NULL, 
    adr_line_2 VARCHAR(40),
    city VARCHAR(29) NOT NULL,
    state CHAR(2) NOT NULL, 
    postal_cd char(5) NOT NULL,
    lat FLOAT NOT NULL, 
    long FLOAT NOT NULL,
    cnt_code CHAR(2) NOT NULL,
    arr_date DATE NOT NULL,
    dep_date DATE NOT NULL,
    pmt_amt FLOAT NOT NULL
) '''

print(drop)
print(create)

try:
    
    cursor = connection.cursor()
    cursor.execute(drop)
    print('Dropped reservations table')
    
    cursor = connection.cursor()
    cursor.execute(create)
    print('Created reservations table')
    
    connection.commit() # commit the changes 
except Exception as e:
    print("Error occurred while creating reservations table:", e)

<psycopg.Connection [IDLE] (host=10.42.160.9 database=postgres) at 0x7fbcbe07f340>
DROP TABLE IF EXISTS reservations
CREATE TABLE reservations(
    res_id SERIAL PRIMARY KEY,
    cust_id INTEGER NOT NULL,
    prp_nm VARCHAR(50) NOT NULL,
    prp_ch VARCHAR(50) NOT NULL,
    adr_line_1 VARCHAR(40) NOT NULL, 
    adr_line_2 VARCHAR(40),
    city VARCHAR(29) NOT NULL,
    state CHAR(2) NOT NULL, 
    postal_cd char(5) NOT NULL,
    lat FLOAT NOT NULL, 
    long FLOAT NOT NULL,
    cnt_code CHAR(2) NOT NULL,
    arr_date DATE NOT NULL,
    dep_date DATE NOT NULL,
    pmt_amt FLOAT NOT NULL
) 
Dropped reservations table
Created reservations table


#### 3. Insert records into the table

In [5]:
import psycopg
from faker import Faker
import random

fake = Faker()
random.seed(0)
Faker.seed(0)

connection = connect()
print(connection)

sql = "INSERT INTO reservations (cust_id, prp_nm, prp_ch, adr_line_1, adr_line_2, city, state, postal_cd, lat,\
       long, cnt_code, arr_date, dep_date, pmt_amt) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,\
       %s, %s)"

records = []

# we only select the 30 countries from the seed used to generate the currency table (referential integrity)
country_codes = []
while (len(country_codes) < 30):
    l = fake.country_code()
    if (l not in country_codes):
        country_codes.append(l)
print(country_codes)
    
for i in range(100):
    cust_id = random.randint(1,50)
    pNm = fake.company() 
    pCh = fake.company()
    city = fake.city()
    state = fake.state_abbr()
    pCode = fake.postalcode() 
    lat = fake.latitude()
    long = fake.longitude()
    cCode =  country_codes[random.randint(0,29)]
    aDate = fake.date_this_year()
    dDate = fake.future_date()
    pmt_amt = round(random.uniform(50, 500),2) # price range of reservations
    
    
    add = fake.street_address()
    st1 = ""
    st2 = ""
    
    if "Suite" in add:
        m = add.split("Suite ")
        st1 = m[0]
        st2 = "Suite " + m[1]
    elif "Apt." in add:
        m = add.split("Apt. ")
        st1 = m[0]
        st2 = "Apt. " + m[1]
    elif "Box" in add:
        m = add.split(", Box ")
        st1 = m[0]
        st2 = "Box " + m[1]
    else:
        st1 = add
        st2 = None
        
    rec = (cust_id, pNm, pCh, st1, st2, city, state, pCode, lat, long, cCode, aDate, dDate, pmt_amt)
    records.append(rec)

try:
    cursor = connection.cursor()
    
    for rec in records:
        cursor.execute(sql, rec)
    
    connection.commit()
    print("Wrote 100 records into reservations table")
        
except Exception as e:
    print("Failed to insert record into person table:", e)

<psycopg.Connection [IDLE] (host=10.42.160.9 database=postgres) at 0x7fbcb7c0eb60>
['MV', 'PS', 'NL', 'BB', 'IL', 'SE', 'RU', 'NA', 'KI', 'QA', 'MK', 'TV', 'GM', 'SD', 'CU', 'JM', 'TW', 'BY', 'VN', 'ID', 'SN', 'MA', 'UY', 'CY', 'KR', 'BZ', 'ZA', 'BN', 'LA', 'LS']
Wrote 100 records into reservations table
